#### Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from dtw import dtw # dynamic timewarping from dtw-python package

from scipy.interpolate import interp1d # for time interpolation

from Sports2D import Sports2D

# importing custom plotting functions
from plotting_functions import plot_joint_angle, plot_all_joints, plot_hip_trajectory

from utilities import list_filepaths, load_multiple_files, compare_joint_angles, compare_aggregated_joint_angles

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



#### Analysing the video's

In [2]:
# Set this to true if you need to re-analyse the videos :)
run_sports2d_analysis = False

In [3]:
import os
from Sports2D import Sports2D

# Define paths relative to the current script/notebook directory
script_dir = os.getcwd()

video_dir = os.path.normpath(os.path.join(script_dir, "../Videos to Analyze/Backflip")) # switch to /Start Salto
config_path = os.path.normpath(os.path.join(script_dir, "../Configs/backflip_config.toml")) # still need to manually move analyzed videos and data to their respective folders (or change it in config)
data_dir = os.path.normpath(os.path.join(script_dir, "../Analyzed Data"))

# Specific move directories
st_salto_data_dir = os.path.normpath(os.path.join(data_dir, "../Analyzed Data/Start Salto"))
backflip_data_dir = os.path.normpath(os.path.join(data_dir, "../Analyzed Data/Backflip"))


# Get a list of all video files in the directory
video_files = list_filepaths(video_dir, [".mp4", ".mov"])

# Load the configuration file
config = Sports2D.read_config_file(config_path)


In [4]:
# Update the video input in the config and process each video

if run_sports2d_analysis == True:
    for video_file in video_files:
        try:
            config['project']['video_input'] = [video_file]
            Sports2D.process(config)
        except Exception as e:
            print(f"Error processing video {video_file}: {e}")

#### Loading Data

In [5]:
# get file paths contained in analyzed video data
backflip_trc_file_paths = list_filepaths(backflip_data_dir, '.trc')
backflip_mot_file_paths = list_filepaths(backflip_data_dir, '.mot')

#print(backflip_trc_file_paths)

# create combined dataframes of them
backflip_trc_combined_df = load_multiple_files(backflip_trc_file_paths, file_type='trc')
backflip_mot_combined_df = load_multiple_files(backflip_mot_file_paths, file_type='mot')

For filepath: /Users/niels/Desktop/University/Third Semester/Perception and Action/Exam/Gymnastics Motion Tracking/Code for Gym Tracking/Analyzed Data/Backflip/id1_2_baseline_Sports2D/id1_2_baseline_Sports2D_px_person00.trc, Match status: <re.Match object; span=(161, 205), match='Sports2D/id1_2_baseline_Sports2D_px_person00'>
For filepath: /Users/niels/Desktop/University/Third Semester/Perception and Action/Exam/Gymnastics Motion Tracking/Code for Gym Tracking/Analyzed Data/Backflip/id1_3_pettlep_Sports2D/id1_3_pettlep_Sports2D_px_person00.trc, Match status: <re.Match object; span=(160, 203), match='Sports2D/id1_3_pettlep_Sports2D_px_person00'>
For filepath: /Users/niels/Desktop/University/Third Semester/Perception and Action/Exam/Gymnastics Motion Tracking/Code for Gym Tracking/Analyzed Data/Backflip/id1_1_baseline_Sports2D/id1_1_baseline_Sports2D_px_person00.trc, Match status: <re.Match object; span=(161, 205), match='Sports2D/id1_1_baseline_Sports2D_px_person00'>
For filepath: /User

#### Combine into 1 dataframe

In [8]:
# join trc and mot
# Ensure consistent column naming
backflip_trc_combined_df = backflip_trc_combined_df.rename(columns={"Time": "time"})

# Perform the merge
merged_df = pd.merge(
    backflip_trc_combined_df,
    backflip_mot_combined_df,
    on=["time", "participant_id", "video_number", "condition", "person_tracked"],
    how="inner"  # Only include rows with matching keys
)

# Optional: Drop redundant columns from `mot` DataFrame
merged_df = merged_df.drop(columns=["version", "nRows", "nColumns", "inDegrees"], errors="ignore")

# Check the results
print(f"Shape of merged DataFrame: {merged_df.shape}")
print(f"Columns in merged DataFrame: {merged_df.columns.tolist()}")
print(f"Unique participants: {merged_df['participant_id'].nunique()}")
print(f"Unique video numbers: {merged_df['video_number'].nunique()}")
print(f"Unique conditions: {merged_df['condition'].nunique()}")

Shape of merged DataFrame: (1210, 95)
Columns in merged DataFrame: ['Frame#', 'time', 'Hip_X', 'Hip_Y', 'Hip_Z', 'RHip_X', 'RHip_Y', 'RHip_Z', 'RKnee_X', 'RKnee_Y', 'RKnee_Z', 'RAnkle_X', 'RAnkle_Y', 'RAnkle_Z', 'RBigToe_X', 'RBigToe_Y', 'RBigToe_Z', 'RSmallToe_X', 'RSmallToe_Y', 'RSmallToe_Z', 'RHeel_X', 'RHeel_Y', 'RHeel_Z', 'LHip_X', 'LHip_Y', 'LHip_Z', 'LKnee_X', 'LKnee_Y', 'LKnee_Z', 'LAnkle_X', 'LAnkle_Y', 'LAnkle_Z', 'LBigToe_X', 'LBigToe_Y', 'LBigToe_Z', 'LSmallToe_X', 'LSmallToe_Y', 'LSmallToe_Z', 'LHeel_X', 'LHeel_Y', 'LHeel_Z', 'Neck_X', 'Neck_Y', 'Neck_Z', 'Head_X', 'Head_Y', 'Head_Z', 'Nose_X', 'Nose_Y', 'Nose_Z', 'RShoulder_X', 'RShoulder_Y', 'RShoulder_Z', 'RElbow_X', 'RElbow_Y', 'RElbow_Z', 'RWrist_X', 'RWrist_Y', 'RWrist_Z', 'LShoulder_X', 'LShoulder_Y', 'LShoulder_Z', 'LElbow_X', 'LElbow_Y', 'LElbow_Z', 'LWrist_X', 'LWrist_Y', 'LWrist_Z', 'participant_id', 'video_number', 'condition', 'person_tracked', 'right_knee', 'left_knee', 'left_ankle', 'right_ankle', 'left_hip'

## Scoring the backflips